# Getting Started With the SHFQA+

## Device Setup and Data Server

Let's start by creating a `DeviceSetup` with a single SHFQA+ instrument. To learn more about what a `DeviceSetup` is and how to use it, have a look at [this page](https://docs.zhinst.com/labone_q_user_manual/core/functionality_and_concepts/00_device_setup/concepts/index.html).

First, we import `laboneq.simple`, which contains the `DeviceSetup`.

In [ ]:
from laboneq.simple import *

Create the `DeviceSetup` with the information about the data server it should connect to. Here, we use "localhost". Change this to the LabOne data server that is running on your computer. You can find the IP address of LabOne data server by navigating to Config > Data Server > Host in the LabOne UI.

Note that calling `add_dataserver` below does not attempt to make the connection yet. This will be done only upon calling `Session.connect()`.

In [ ]:
device_setup = DeviceSetup("ZI_SHFQA+")
device_setup.add_dataserver(
    host="localhost",
    port="8004",
)

Create an `SHFQA` instrument, which was imported from `laboneq.simple`, and add it to the `DeviceSetup`. 

When creating the instrument instance, you need to specify the device ID under `address`; for example, `dev12345`.

If you do not have an active LabOne data server running to connect to the instrument, you also need to specify the `device_options` that are installed on your instrument. These options are used to ensure a correct experiment compilation for your system.

The possible options you can set for an SHFQA instrument are:
* either `"SHFQA4"` if you have a 4-channel instrument or `"SHFQA2"` for a 2-channel version;
* `PLUS` if you have the SHFQA+;
* `"16W"` available only in combination with `"SHFQA2"`to extend the avilable integration weights to 16 per channel.

When passing these options to the instrument, add them in any order, separated by a forward slash (`"/"`). Below, we will use `device_options="SHFQA4/PLUS"`. Have a look at [this page](https://docs.zhinst.com/labone_q_user_manual/core/functionality_and_concepts/00_device_setup/concepts/01_instrument_options.html) to learn more about the possible options of our instruments.

You can also pass additional input parameters to configure your instrument:
  
* the `interface` over which to connect to the instrument, either `"1GbE"` (default) or `"usb"`. Note that **to ensure the stability of the connection to the instrument, we recommend to use the ethernet interface instead of USB**;

* the `reference_clock_source` for the instrument, either as `"internal"` to use the instrument's own internal reference clock, or `"external"` (default) if you are using an external source like the PQSC instrument, for example.

In [ ]:
shfqa = SHFQA(
    uid="shfqa",
    address="dev12345",
    interface="1GbE",
    device_options="SHFQA4/PLUS",
    reference_clock_source="internal",
)

device_setup.add_instruments(shfqa)

Next, we create connections to each of the 4 ports of the instrument and add these connections to the `DeviceSetup`. These connections are represented in LabOne Q as [logical signal lines](https://docs.zhinst.com/labone_q_user_manual/core/functionality_and_concepts/02_logical_signals/concepts/index.html) between the instruments and your device under test. Here, we assume the device under test is a four-qubit QPU and use the physical lines of these qubits as a naming convention for our signal lines. Note that the instances of the `LogicalSignal`s will be created automatically by the `add_connections` method of `DeviceSetup`, with the names that we have specified.

In [ ]:
# Create measure and acquire lines for the 4 qubits connected to the 4 QA output and input ports of the instrument.

for idx in range(4):
    device_setup.add_connections(
        "shfqa",
        create_connection(
            to_signal=f"q{idx}/measure", ports=f"QACHANNELS/{idx}/OUTPUT", type="iq"
        ),
        create_connection(
            to_signal=f"q{idx}/acquire", ports=f"QACHANNELS/{idx}/INPUT", type="acquire"
        ),
    )

You can inspect the `LogicalSignal`s that have been created by calling `device_setup.logical_signal_by_uid(signal_name)`; for example:

In [ ]:
device_setup.logical_signal_by_uid("q0/measure")

Next, we need to configure the `calibration` of the signal lines of the `DeviceSetup`. We will set the most common properties:

* `range` - the power range in dBm of the input and output ports;
* `local_oscillator` - an instance of `Oscillator` where we specify the local oscillator frequency;
* `oscillator` - an instance of `Oscillator` where we specify the IF frequency that will be mixed with local oscillator frequency.

Keep in mind that the channels 0-1 and 2-3 share the local oscillator, so the local oscillator for these channels must be configured with the same frequency value.

In [ ]:
qa_lo_0 = Oscillator(frequency=7e9)
qa_lo_1 = Oscillator(frequency=7.2e9)

config = Calibration()
for idx, qa_lo in enumerate([qa_lo_0, qa_lo_1]):
    config[f"q{2 * idx}/measure"] = SignalCalibration(
        local_oscillator=qa_lo, oscillator=Oscillator(frequency=100e6), range=5
    )
    config[f"q{2 * idx}/acquire"] = SignalCalibration(
        local_oscillator=qa_lo, oscillator=Oscillator(frequency=100e6), range=10
    )
    config[f"q{2 * idx + 1}/measure"] = SignalCalibration(
        local_oscillator=qa_lo, oscillator=Oscillator(frequency=100e6), range=5
    )
    config[f"q{2 * idx + 1}/acquire"] = SignalCalibration(
        local_oscillator=qa_lo, oscillator=Oscillator(frequency=100e6), range=10
    )

# Apply the configuration to the DeviceSetup
device_setup.set_calibration(config)

Great! We have our `DeviceSetup` for a single SHFQA+ instrument.

Before we can play a signal on the instrument, we first have to connect it to the LabOne data server via the LabOne Q [Session](https://docs.zhinst.com/labone_q_user_manual/core/functionality_and_concepts/01_session/concepts/index.html). Here, we connect in emulation mode by calling `Session.connect` with `do_emulation=True`. Set this flag to False in order to connect to a physical setup.

In [ ]:
session = Session(device_setup)
session.connect(do_emulation=True)  # do_emulation=False when at a physical setup

## Generate a Simple Experiment

Let's create a simple experiment that plays back a 2-$\mu$s gaussian-square pulse on each of the output ports of the SHFQA+, and run an integrated average acquisition using a constant 2-$\mu$s integration kernel. In addition, we sweep the amplitude of the pulses that are output.

To learn more about the `Experiment` object and how to write experiments in LabOne Q, have a look at the ["Experiment Definition"](https://docs.zhinst.com/labone_q_user_manual/core/functionality_and_concepts/05_experiment/concepts/index.html) and ["Writing and Experiment Workflow"](https://docs.zhinst.com/labone_q_user_manual/applications_library/tutorials/sources/writing_experiments.html#write-the-experiment-pulse-sequence) sections of the manual.

In [ ]:
import numpy as np


@dsl.experiment(
    signals=[f"q{idx}_measure" for idx in range(4)]
    + [f"q{idx}_acquire" for idx in range(4)]
)
def simple_experiment(count):
    with dsl.sweep(
        name="amplitude_sweep",
        parameter=SweepParameter("measure_pulse_amplitudes", np.linspace(0, 1, 7)),
    ) as amplitude:
        with dsl.acquire_loop_rt(
            count=count,
            averaging_mode=AveragingMode.CYCLIC,
            acquisition_type=AcquisitionType.INTEGRATION,
        ):
            for idx in range(4):
                with dsl.section(name=f"play-measure-pulse-and-acquire-q{idx}"):
                    dsl.play(
                        f"q{idx}_measure",
                        pulse_library.gaussian_square(amplitude=1, length=2e-6),
                    )
                    dsl.acquire(
                        f"q{idx}_acquire",
                        handle=f"q{idx}/result",
                        kernel=pulse_library.const(amplitude=1, length=2e-6),
                    )
                    # add a processing delay
                    dsl.delay(f"q{idx}_acquire", time=0.5e-6)

    # Set the amplitude sweep in the experiment calibration to sweep the gain of the output channel
    exp_calibration = dsl.experiment_calibration()
    for idx in range(4):
        exp_calibration[f"q{idx}_measure"] = SignalCalibration(amplitude=amplitude)

        # Map the ExperimentSignals "q{idx}_measure", "q{idx}_acquire" to the logical signal lines defined in the `DeviceSetup`
        dsl.map_signal(f"q{idx}_measure", f"q{idx}/measure")
        dsl.map_signal(f"q{idx}_acquire", f"q{idx}/acquire")

Next, we instantiate the `Experiment` by running the function `simple_experiment`.

In [ ]:
experiment = simple_experiment(5)

Note that, the mapping between the `ExperimentSignal`s and the logical signal lines do not have to be done as part of `simple_experiment` but can also be done or modified on the `Experiment` instance returned by `simple_experiment` as follows:

```python
q0_ls = device_setup.logical_signal_groups["q0"].logical_signals
experiment.map_signal("measure", q0_ls["measure"])
experiment.map_signal("acquire", q0_ls["acquire"])
```

You can also map an `ExperimentSignal` directly to a `LogicalSignal` instead of its UID, as written above.

Now we compile the experiment and inspect it using `plot_simulation`. Note that we will not see the amplitude changing in the simulation below. In this case, the amplitude is being swept by changing the linear gain factor on the output line of the SHFQA while the amplitude of the pulse generated by the AWG (what is plotted by the simulator) remains constant.

In [ ]:
compiled_experiment = session.compile(experiment)

In [ ]:
from laboneq.contrib.example_helpers.plotting.plot_helpers import plot_simulation

plot_simulation(compiled_experiment, start_time=0, length=15e-6)

You can inspect your experiment in more detail by using the interactive pulse-sheet viewer. Calling the function `show_pulse_sheet` creates an HTML file that you can open in your browser. To show the pulse sheet in the kernel, set `interactive=True`.

In [ ]:
show_pulse_sheet("simple_pulse_sequence", compiled_experiment, interactive=True)

Finally, let's run the experiment on the instrument. 

In [ ]:
results = session.run()

You can inspect the acquired results using the handle name that we've specified when defining the pulse sequence, `q0/result`:

In [ ]:
results["q0/result"]

Or, alternatively,

In [ ]:
results.q0.result